In [2]:
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd

In [3]:
MLFLOW_TRACKING_URI = 'sqlite:///../cred_risk_sqlite_aws_mlflow.db'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
experiments = client.search_experiments()

for exp in experiments:
    print(f"Name: {exp.name}, ID: {exp.experiment_id}, Artifact Location: {exp.artifact_location}")

Name: credit_default_risk_experiment_tracking_aws, ID: 2, Artifact Location: s3://mlflow-credit-default-risk-prediction-artifact-store-v2
Name: Default, ID: 0, Artifact Location: s3://mlflow-credit-default-risk-prediction-artifact-store-v2/0


In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = '2',
    filter_string = 'metrics.auc > 0.732',
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ['metrics.auc ASC']
)

In [6]:
for run in runs:
    print(f"runId: {run.info.run_id}, auc: {run.data.metrics['auc']:.4f}")

runId: fe69b7b9817240789feb57c59ff31cc5, auc: 0.7330


In [7]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [16]:
run_id = 'fe69b7b9817240789feb57c59ff31cc5'
model_uri = f'runs:/{run_id}/models/xgboost_model'

In [11]:
mlflow.register_model(model_uri=model_uri,name='credit_default_risk_xgb_model_v2')

Successfully registered model 'credit_default_risk_xgb_model_v2'.
Created version '1' of model 'credit_default_risk_xgb_model_v2'.


<ModelVersion: aliases=[], creation_timestamp=1754828821455, current_stage='None', description=None, last_updated_timestamp=1754828821455, name='credit_default_risk_xgb_model_v2', run_id='fe69b7b9817240789feb57c59ff31cc5', run_link=None, source='s3://mlflow-credit-default-risk-prediction-artifact-store-v2/fe69b7b9817240789feb57c59ff31cc5/artifacts/models/xgboost_model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [13]:
model_name = 'credit_default_risk_xgb_model_v1'
latest_versions = client.get_latest_versions(name=model_name)
    
for version in latest_versions:
    print(f'version: {version.version}, stage: {version.current_stage}')

version: 1, stage: None
version: 2, stage: Production


/var/folders/gy/0jy_9kfn7c5b6pxfsksq3m8w0000gn/T/ipykernel_23358/3326168028.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [13]:
model_version = 2
new_stage = 'Staging'

In [9]:
model_version = 2
new_stage = 'Production'

In [10]:
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/var/folders/gy/0jy_9kfn7c5b6pxfsksq3m8w0000gn/T/ipykernel_23358/227647614.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1754828711806, current_stage='Production', description='model version 2 was transitioned to Staging on 2025-08-10 13:28:08.728142', last_updated_timestamp=1754829857954, name='credit_default_risk_xgb_model_v1', run_id='fe69b7b9817240789feb57c59ff31cc5', run_link='', source='models:/credit_default_risk_xgb_model_v1/1', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [11]:
from datetime import datetime as dt

In [12]:
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f'model version {model_version} was transitioned to {new_stage} on {dt.today().now() }'
)

<ModelVersion: aliases=[], creation_timestamp=1754828711806, current_stage='Production', description='model version 2 was transitioned to Production on 2025-08-10 13:44:25.206935', last_updated_timestamp=1754829865209, name='credit_default_risk_xgb_model_v1', run_id='fe69b7b9817240789feb57c59ff31cc5', run_link='', source='models:/credit_default_risk_xgb_model_v1/1', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [18]:
client.download_artifacts(run_id=run_id, path='', dst_path='../')

'/Users/mac/Projects/MLops_credit_default_risk_prediction/'

In [ ]:
import pickle

with open('models/model.xgb', 'rb') as f_in:
    dv = pickle.load(f_in)